In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Loading the dataset

In [4]:
data = pd.read_csv('IMDB Dataset.csv')

In [11]:
data.shape

(50000, 2)

In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [14]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [15]:
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)
     

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### Data preprocessing

In [21]:
# Tokenize test data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)
     

In [17]:
#split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [18]:
print(X_train)
     

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [19]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']
     

In [6]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [20]:
print(Y_train)
     

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


### Long short term memory

In [8]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
     

C:\Users\smrut\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the model

In [23]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)
     

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 91s 175ms/step - accuracy: 0.7287 - loss: 0.5299 - val_accuracy: 0.8495 - val_loss: 0.3549
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 106s 211ms/step - accuracy: 0.8516 - loss: 0.3566 - val_accuracy: 0.8381 - val_loss: 0.3729
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 130s 260ms/step - accuracy: 0.8676 - loss: 0.3245 - val_accuracy: 0.8706 - val_loss: 0.3125
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 287ms/step - accuracy: 0.8825 - loss: 0.2926 - val_accuracy: 0.8474 - val_loss: 0.3788
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 135s 270ms/step - accuracy: 0.8647 - loss: 0.3321 - val_accuracy: 0.8646 - val_loss: 0.3415


In [24]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.8703 - loss: 0.3316
Test Loss: 0.3263346552848816
Test Accuracy: 0.8726000189781189


### Building a predictive model

In [25]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
  return sentiment
     

In [26]:
# example usage
new_review = 'This movie was fantastic. I loves it.'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')
     

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
The sentiment of the review is: positive


In [27]:
new_review = 'This Movie was not that good'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
The sentiment of the review is: negative
